# SEED - ROI strength vs distance 
All the plots below are between ROIs that have at least a single seed connecting them by a strength larger than 2500 (Number of connected streamlines).  
The plots below are in three groups each containing 8 subplots.  
 - GROUP 1: ROIs are very close
 - GROUP 2: ROIs are midrange 
 - GROUP 3: ROIs are very Far

And the study is only for ROIs in right hemispher.

# Envelope Regression
Regression is based on Donahue model
 - Donahue model ```Log(NOS) = A + B*distance ```

In order to regress the above model, first we form regression set ```R``` in space **S=(distance,NOS)** space, and then use regular least square method to fit the model into ```R``` .

> **Right(p)** for each point p in S is defined as subset of points in S with larger __distance__ than p  

> **Right_Neighbors(p,k)** for each point p in S is defined the k first points to the right of the p in S with larger __distance__ than p   

Each point __p__ in **S** is also in ```R``` if any of the below two criterias are met:  
1 - __p__ has larger NOS than all points in **Right(p)**  
2 - __p__ has larger NOS than all points in **Right_Neighbors(p,5%)**

In finding **Right_Neighbors(p,5%)** above, the algorithm restricts the maximum distance between points to ```p.distance*15/100+5```, i.e initailly the maximum bon width for K-NN is set to 5 but it grows linearly for larger distances.
 
# Plots
 Each plot shows:
     1. Regressor line (red)
     2. Distance-Strength points (Blue Circle)
     3. Distance-Strength points used in regression i.e, R (Big green circle)
     4. Neighborhood of points in R are plotted by horizontal lines
     
> K is set to be equal to 5% of all points.  

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from py2neo import Node, Relationship, Graph
from numpy.random import rand
import numpy as num
from bokeh.plotting import output_notebook,figure, show
from bokeh.layouts import row,column,gridplot
from bokeh.models import Label
import numpy as np
import matplotlib.pyplot as plt
from curve import *
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

# Group 1 - ROIs are close

In [3]:
# NEO4J database call for GROUP 1 - Close and Connected
graph = Graph()
c1 = '''MATCH (n:ROI)-[r:NOS]->(m:ROI)
WHERE r.weight > 2500 and r.length>1 and n.hemisphere="L"
RETURN n.name AS n1 ,m.name AS n2,r.length as D,r.weight as W,r._length,r._weight
ORDER BY r.length
limit 8'''


A = graph.run(c1)
# Plotting the data
s1 = []
s2 = []
param1 = []
param2 = []
for i,w in enumerate(A):
#     p2 = figure(plot_width=500, plot_height=500, title=w['n1']+'->'+w['n2'],y_axis_label = "Strength",x_axis_label = "Distance")
    a = w['r._length'] 
    b = w['r._weight']
    c = sorted(zip(a,b))
    d = [xx for xx in c if (xx[1]>200 and xx[0]>1)]
    a = [xx[0] for xx in d]
    b = [np.log(xx[1]) for xx in d]
    D = maxBin2(a,b,10)
    outs = D['outs']
    bins = D['bins']
    l = len(outs)
    p1 = figure(plot_width=500, plot_height=500, title=w['n1']+'->'+w['n2']+'(R2%='+str(D['r'])+')',y_axis_label = "Strength(log scale)",x_axis_label = "Distance")
    p1.circle(a,b)
    p1.line(D['x'],D['z'],legend="Donahue",color="red")
#     p1.line(D['cxw'],D['czw'],legend="Donahue - weighted regression",color="blue")
    for i in range(l):
            p1.circle_cross(outs[i][0][0],outs[i][0][1],color='green',size=8)
    de = num.mean(b)
    for j,w in enumerate(D['bins']):
        dec = num.random.rand()
        se = outs[j][0][1]
        if dec < .3:
            c= "yellow"
        elif dec<.6:
            c = "cyan"
        else:
            c="black"
        p1.line([w[0],w[1]],[se,se],color=c)
    s1 = s1 + [p1]
grid = gridplot([[s1[0], s1[1]], [s1[2], s1[3]], [s1[4], s1[5]], [s1[6], s1[7]]])
show (grid)
print(D['r'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/minpack.py:785: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


96.0


# Midrange and Connected

In [4]:
# NEO4J database call for GROUP 1 - Close and Connected
graph = Graph()

c1 = '''MATCH (n:ROI)-[r:NOS]->(m:ROI)
WHERE r.weight > 2500 and r.length>20
RETURN n.name AS n1 ,m.name AS n2,r.length as D,r.weight as W,r._length,r._weight
ORDER BY r.length
limit 8'''

# c1 = '''MATCH (n:ROI)-[r:NOS]->(m:ROI)
# WHERE r.weight > 2500 and r.length>1
# RETURN n.name AS n1 ,m.name AS n2,r.length as D,r.weight as W,r._length,r._weight
# ORDER BY r.length DESC
# limit 8'''

A = graph.run(c1)
# Plotting the data
s1 = []
s2 = []
param1 = []
param2 = []
for i,w in enumerate(A):
    a = w['r._length'] 
    b = w['r._weight']
    c = sorted(zip(a,b))
    d = [xx for xx in c if (xx[1]>200 and xx[0]>1)]
    a = [xx[0] for xx in d]
    b = [np.log(xx[1]) for xx in d]
    D = maxBin2(a,b,10)
    outs = D['outs']
    bins = D['bins']
    l = len(outs)
    p1 = figure(plot_width=500, plot_height=500, title=w['n1']+'->'+w['n2']+'(R2%='+str(D['r'])+')',y_axis_label = "Strength(log scale)",x_axis_label = "Distance")
    p1.circle(a,b)
    p1.line(D['x'],D['z'],legend="Donahue",color="red")
#     p1.line(D['cxw'],D['czw'],legend="Donahue - weighted regression",color="blue")
    for i in range(l):
            p1.circle_cross(outs[i][0][0],outs[i][0][1],color='green',size=8)
    de = num.mean(b)
    for j,w in enumerate(D['bins']):
        dec = num.random.rand()
        se = outs[j][0][1]
        if dec < .3:
            c= "yellow"
        elif dec<.6:
            c = "cyan"
        else:
            c="black"
        p1.line([w[0],w[1]],[se,se],color=c)

    s1 = s1 + [p1]
grid = gridplot([[s1[0], s1[1]], [s1[2], s1[3]], [s1[4], s1[5]], [s1[6], s1[7]]])
show (grid)

# Group 2 - Far and connected

In [5]:
# NEO4J database call for GROUP 1 - Close and Connected
graph = Graph()


c1 = '''MATCH (n:ROI)-[r:NOS]->(m:ROI)
WHERE r.weight > 2500 and r.length>1
RETURN n.name AS n1 ,m.name AS n2,r.length as D,r.weight as W,r._length,r._weight
ORDER BY r.length DESC
limit 8'''

A = graph.run(c1)
# Plotting the data
s1 = []
s2 = []
param1 = []
param2 = []
for i,w in enumerate(A):
    a = w['r._length'] 
    b = w['r._weight']
    c = sorted(zip(a,b))
    d = [xx for xx in c if (xx[1]>200 and xx[0]>1)]
    a = [xx[0] for xx in d]
    b = [np.log(xx[1]) for xx in d]
    D = maxBin2(a,b,10)
    outs = D['outs']
    bins = D['bins']
    l = len(outs)
    p1 = figure(plot_width=500, plot_height=500, title=w['n1']+'->'+w['n2']+'(R2%='+str(D['r'])+')',y_axis_label = "Strength(log scale)",x_axis_label = "Distance")
    p1.circle(a,b)
    p1.line(D['x'],D['z'],legend="Donahue",color="red")
#     p1.line(D['cxw'],D['czw'],legend="Donahue - weighted regression",color="blue")
    for i in range(l):
            p1.circle_cross(outs[i][0][0],outs[i][0][1],color='green',size=8)
    de = num.mean(b)
    for j,w in enumerate(D['bins']):
        dec = num.random.rand()
        se = outs[j][0][1]
        if dec < .3:
            c= "yellow"
        elif dec<.6:
            c = "cyan"
        else:
            c="black"
        p1.line([w[0],w[1]],[se,se],color=c)

    s1 = s1 + [p1]
grid = gridplot([[s1[0], s1[1]], [s1[2], s1[3]], [s1[4], s1[5]], [s1[6], s1[7]]])
show (grid)